# Handwriting,signatures and initials  detection in contract management

### __Part 1 __

Dans cet Article , nous fournissons un guide pratique montrant comment vous pouvez rapidement créer un ensemble de données et former un modèle de détection d'objet personnalisé pour la détection de signature et intial, et écriture manuscrite .

Pour les entreprises de toutes sortes, l'un des grands avantages du passage des documents physiques aux documents numériques est la rapidité et l'efficacité des méthodes de recherche et d'extraction de connaissances désormais disponibles. Il est révolu le temps d'examiner les documents ligne par ligne pour trouver des informations particulières. Cependant, les choses se compliquent lorsque le chercheur doit extraire des concepts généraux plutôt que des phrases spécifiques. Et c'est encore plus compliqué lorsqu'il est appliqué à des documents numérisés de qualité mixte contenant des annotations manuscrites

Il est difficile d'obtenir une précision d'extraction acceptable lors des l'application de méthodes traditionnelles de recherche et d'extraction de connaissances à ces documents. Le principal de ces défis est la mauvaise qualité d'image des documents et les annotations manuscrites. La mauvaise qualité d'image provient du fait que ces documents sont souvent des copies numérisées des accords signés, stockées sous forme de PDF, souvent une ou deux générations retirées de l'original. Cela provoque de nombreuses erreurs de reconnaissance optique de caractères (OCR) qui introduisent des mots absurdes. De plus, la plupart de ces contrats comprennent des annotations manuscrites qui modifient ou définissent les conditions essentielles de l'accord. La lisibilité, le style et l'orientation de l'écriture varient considérablement; et l'écriture manuscrite peut apparaître n'importe où sur la page de contrat imprimée par machine.

Donc La plupart des méthodes que nous  sont liées au traitement automatique du langage naturel (NLP). Dans cet article, nous visons à montrer comment nous exploitons également des algorithmes de vision par ordinateur pour améliorer l'analyse des contrats et soutenir notre pipeline NLP là où il en manque.

## Énoncé du problème technique.


Malgré les progrès récents, la technologie OCR standard fonctionne mal pour reconnaître les caractères manuscrits sur une page imprimée par machine. La précision de la reconnaissance varie considérablement pour les raisons décrites ci-dessus, et le logiciel égare souvent l'emplacement des informations manuscrites lorsqu'il la fait correspondre au texte adjacent. Bien que les logiciels de reconnaissance d'écriture manuscrite ont longtemps été des applications autonomes, il existe peu de solutions qui fonctionnent bien avec l'OCR des documents et les pipelines de recherche.

Afin de permettre l'extraction d'entités et de connaissances à partir de documents avec des annotations manuscrites, l'objectif de notre solution était d'abord d'identifier les mots manuscrits sur une page imprimée, puis de reconnaître les caractères pour transcrire le texte, et enfin de réinsérer ces caractères reconnus dans l'OCR résultat au bon endroit. Pour une bonne expérience utilisateur, tout cela devrait être intégré de manière transparente dans le flux de travail d'ingestion de documents. Et aussi il convient également de tirer parti des éléments visuels pour donner un sens aux sections non textuelles telles que les signatures ou les paraphes. 

## Approche
Ces dernières années, les modèles de détection d'objets de vision par ordinateur utilisant des réseaux de neurones profonds se sont révélés efficaces pour une grande variété de tâches de reconnaissance d'objets, mais nécessitent une grande quantité de données d'entraînement expertement étiquetées. Heureusement, les modèles pré-formés sur des ensembles de données standard tels que COCO , contenant des millions d'images étiquetées, peuvent être utilisés pour créer de puissants détecteurs personnalisés avec des données limitées via l'apprentissage par transfert, une méthode pour affiner un modèle existant pour accomplir une tâche différente mais connexe . Il a été démontré que l'apprentissage par transfert réduit considérablement la quantité de données de formation requises pour atteindre une bonne précision pour un large éventail d'applications .

Dans ce cas particulier, le transfert d'apprentissage à partir d'un modèle pré-formé était un choix évident, compte tenu de notre petit échantillon d'annotations manuscrites étiquetées et signature .. .

Notre flux de travail, de la détection d'objets à la reconnaissance et au remplacement de l'écriture manuscrite dans le résultat OCR de l'image du contrat, est résumé dans la figure 1 ci-dessous. Pour commencer, nous avons appliqué un modèle de détection d'objet personnalisé sur une image d'une page imprimée de contrat pour détecter l'écriture manuscrite et identifier son cadre de sélection, aussi pour les signature et paraphes.


<img src="data2/signature.png" width="600"/>


Nous utiliserons des algorithmes de détection d'objets, qui peuvent détecter et étiqueter plusieurs objets sur la même image. Les sorties sont des boîtes englobantes situées autour de l'objet d'intérêt. Nous tirons parti d'un modèle basé sur un réseau de neurones convolutionnel convolutif ( Faster RCNN ) pré-formé sur le jeu de données (COCO) et affinons notre tâche de détection. fast_rcnn_resnet50_coco_2018_01_28, un modèle formé sur l' ensemble de données Coco contenant plus de 200 000 images étiquetées et 1,5 million d'instances d'objets réparties dans 80 catégories.

Pour notre application personnalisée, on vas utiliser localement le logiciel open source (VOTT) pour étiqueter manuellement un petit ensemble de données sur les marchés publics  contenant à la fois du texte imprimé par machine et de l'écriture manuscrite et signatures et paraphes  , comme nous le détaillerons dans la section des données ci-dessous. Nous avons étiqueté trois classes d'objets d'écriture manuscrite dans l'outil VOTT les signatures et les non-signatures (texte général comme les dates) et paraph, enregistrant la boîte englobante et l'étiquette pour chaque instance.

Une fois que nous avons reconnu les annotations manuscrites, nous avons utilisé l'OCR afin de reconnaître les caractères de l'écriture manuscrite. Vous pouvez trouver les cahiers Jupyter pour ce projet, et un échantillon des données sur le référentiel GitHub du projet .

## Les données



Notre ensemble de données est composé d'une variété de contrats du gouvernement télécharger depuis ce lien https://www.gsa.gov/real-estate/real-estate-services/leasing-policy-procedures/lease-documents .

L'utilisation de VOTT nous a permis de produire un ensemble de formation de 182 images étiquetées à partir d'un échantillon de contrats du gouvernement en quelques heures. Nous avons tiré notre ensemble de tests de 100 images contractuelles supplémentaires.

Comme décrit dans l'approche, nous avons étiqueté trois classes: les signatures et l'écriture manuscrite et les intials . Notre objectif était principalement d'interpréter correctement les objets écritures manuscrites, car ils étaient pertinents pour les entités et les concepts que nous essayions d'extraire.

Pou les signatures les intials objectif est de localiser où se trouve la signature dans le document, plus que d'avoir un cadre de délimitation précis.

La classification de l'écriture manuscrite et de signatureet intials en tant que des classes différentes nous a permis de nous concentrer sur l'écriture manuscrite sans signature (resorque)  et signature sans intials ( resporque).

La sortie de VOTT écrit un fichier XML pour chaque image au format Pascal-VOC, avec des informations d'emplacement du cadre de délimitation pour chaque objet étiqueté.


## Étape 1: annoter des images
L'étape suivante consiste à annoter les contrats, nous avons utilisé localement le logiciel open source VOTT développé par Microsoft, pour annoter les contrats. D'autres logiciels d'annotation d'image (par exemple Labelbox , VGG Image Innotator ) sont disponibles.

<img src="data2/vott.png" width="600"/>

L'utilisation de VOTT est assez intuitive. Après avoir créé un nouveau projet, vous devrez définir une connexion source (d'où charger les actifs) et une connexion cible (où enregistrer le projet et les données exportées) et ajouter les balises ou les étiquettes dont vous avez besoin. Une fois le projet créé et les actifs importés, vous pouvez démarrer l'annotation manuelle. Une fois l'annotation terminée, exportez le projet.

<img src="data2/voot_capt.png" width="800"/>


Dans le chemin que vous avez défini dans la connexion cible, vous trouverez le fichier «NAME_OF_YOUR_PROJECT-export.json» contenant toutes les annotations de vos boîtes englobantes. Ce fichier json stocke un identifiant unique pour chaque actif (" 00abf77f552ce71b4a4673ce40fe4479 " dans l'exemple ci-dessous) mappé à un dictionnaire contenant des informations sur l'image (" actif " dans l'exemple ci-dessous) et une liste de toutes les zones de délimitation de la vérité (" régions " sur l'exemple ci-dessous) avec chaque élément de la liste contenant lui-même un dictionnaire stockant des données sur chaque boîte englobante.

<img src="data2/id.PNG" width="500"/>

Une fois les actifs étiquetés, ils peuvent être exportés dans une variété de formats:
    
    
* Azure Custom Vision Service
* Microsoft Cognitive Toolkit (CNTK)
* TensorFlow (Pascal VOC and TFRecords)
* VoTT (generic JSON schema)
* Comma Separated Values (CSV)

De plus, les utilisateurs peuvent choisir d'exporter

* tous les actifs
* actifs visités uniquement
* seuls les éléments balisés

Cliquez sur l' Export (icône flèche) dans la navigation de gauche. Sélectionnez le format  d'exportation approprié et les actifs à exporter. Le pourcentage séparé en ensembles de tests et de formation peut également être ajusté ici.

<img src="data2/dataset.PNG" width="600"/>

Conclustion 

Pour résumer, lors de la création d'annotations avec l'outil Boîte englobante, faites très attention aux paramètres de la boîte englobante que vous créez. Vous devez également vous assurer que les boîtes de délimitation que vous créez ne sont ni trop lâches ni trop serrées. En outre, assurez-vous que les boîtes englobantes reçoivent les annotations appropriées, avec l'étiquette de classe correcte qui leur est appliquée.

Dnas l'article suivant  , On vas utiliser L'API de détection d'objets TensorFlow ou   L'API de détection d'objets de pytorch (Detectron) pour entraîner notre détecteur d'objets, à partir de notre  ensemble de données d'apprentissage.
